In [2]:
import numpy as np
import pandas as pd
import scipy.spatial.distance as dist

In [100]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [202]:
class Agglomerative:
    def __init__(self, points, linkage = 'single', distance='Euclidean', rounding_digit = 2):
        self.points = points
        self.distance = distance
        self.linkage = linkage
        self.rounding_digit = rounding_digit
        

    def cal_distance(self, x1, x2):
        if self.distance == 'Euclidean':
            return round(dist.euclidean(x1,x2),self.rounding_digit)
        elif self.distance == 'Manhattan':
            return sum([abs(i-j) for i,j in zip(x1,x2)])
        
    def find_min(self):
        min_dist = np.inf        
        for i in range(len(self.pt_list)):
            for j in range(len(self.pt_list)):
                if i > j:
                    if min_dist > self.distance_matrix.iloc[i,j]:
                        min_dist = self.distance_matrix.iloc[i,j]
                        col = self.pt_list[i]
                        inx = self.pt_list[j]
        return min_dist, inx, col  
    
    def merge_dist_matrix(self, p1, p2):
        temp_list = [(p1,p2) if x in (p1,p2) else x for x in self.pt_list]
        temp = []
        [temp.append(x) for x in temp_list if x not in temp]
        temp_list = temp
        #print(self.pt_list)
        temp_df = pd.DataFrame(index=temp_list, columns=temp_list)
        temp_df.fillna('', inplace=True)
        #display(temp_df)
        
        for i,inx in enumerate(temp_df.index):
            for j,col in enumerate(temp_df.columns):
                #print(inx, col)
                if i >= j:
                    if inx in self.distance_matrix.index:
                        if col in self.distance_matrix.columns:
                            temp_df.iloc[i,j] = self.get_distance(inx,col)
                        else:
                            dist1 = self.get_distance(inx,col[0])
                            dist2 = self.get_distance(inx,col[1])
#                             print(inx, col)
#                             print(dist1, dist2)
                            temp_df.iloc[i,j] = self.find_linkage_dist(dist1,dist2)
                    elif col in self.distance_matrix.columns:
                        dist1 = self.get_distance(inx[0],col)
                        dist2 = self.get_distance(inx[1],col)
#                         print(inx, col)
#                         print(dist1, dist2)
                        temp_df.iloc[i,j] = self.find_linkage_dist(dist1, dist2)
                    else: 
                        dist1 = self.get_distance(inx[0],col[0])
                        dist2 = self.get_distance(inx[0],col[1])
                        dist3 = self.get_distance(inx[1],col[0])
                        dist4 = self.get_distance(inx[1],col[1])
                        temp_df.iloc[i,j] = self.find_linkage_dist(dist1, dist2, dist3, dist4)
                    #display(temp_df)
        for i in range(len(temp_df.index)):
            for j in range(len(temp_df.columns)):
                if i==j:
                    temp_df.iloc[i,j] = 0
        self.distance_matrix = temp_df
        self.pt_list = temp_list
    
    def get_distance(self, row, col):
        #print(row, col)
        i = self.pt_list.index((row))
        j = self.pt_list.index((col))
        if isinstance(self.distance_matrix.iloc[i,j], float) or isinstance(self.distance_matrix.iloc[i,j], int):
            return self.distance_matrix.iloc[i,j]
        else:
            return self.distance_matrix.iloc[j,i]
    def find_linkage_dist(self, *args):
        #print(args)
        for arg in args:
            if not (isinstance(arg, float) or isinstance(arg, int)):
                return ''
        
        if self.linkage == 'single':
            #print(min(args))
            return min(args)
        if self.linkage == 'complete':
            return max(args)
        else:
            return round(np.mean(args), self.rounding_digit)
        
        
    
    def cluster(self):
        print(f'Points are:\n')
        self.pt_list = [f'P{i+1}' for i in range(len(self.points))]
        self.points_df = pd.Series(self.points, index=self.pt_list)
        print(self.points_df.to_string())
        self.pt_list = [f'P{i+1}' for i in range(len(self.points))]
        self.proximity_matrix = pd.DataFrame(index=self.pt_list, columns=self.pt_list)
        self.proximity_matrix.fillna('', inplace=True)
        
        for i in range(len(self.points)):
            for j in range(len(self.points)):
                if i >= j:
                    self.proximity_matrix.iloc[i,j] = self.cal_distance(self.points[i], self.points[j])
            
        print(f'\nProximity matrix based on {self.distance} distance')
        
        display(self.proximity_matrix)
        self.distance_matrix = self.proximity_matrix
        
        n_iteration = 1
        while self.distance_matrix.shape[0] > 1:
            print(f'\nIteration: {n_iteration}')
            min_dist, inx, col = self.find_min()
            print(f'\nDistance between {inx} and {col} is smallest ({min_dist})')
            print(f'\nMerge {inx} and {col} into a single cluster')
            print(f'\nUsing {self.linkage} linkage, the distance matrix is')
            self.merge_dist_matrix(inx,col)
            #print(self.pt_list)
            display(self.distance_matrix)
            n_iteration +=1
            print('-----------------------------------------------------------------')
            #break
        

In [203]:
Agglomerative([(0.40, 0.53), (0.22, 0.38), (0.35, 0.32), (0.26, 0.19), (0.08, 0.41), (0.45, 0.30)], linkage='average',
              distance='Euclidean').cluster()

Points are:

P1     (0.4, 0.53)
P2    (0.22, 0.38)
P3    (0.35, 0.32)
P4    (0.26, 0.19)
P5    (0.08, 0.41)
P6     (0.45, 0.3)

Proximity matrix based on Euclidean distance


,P1,P2,P3,P4,P5,P6
P1,0.0,,,,,
P2,0.23,0.0,,,,
P3,0.22,0.14,0.0,,,
P4,0.37,0.19,0.16,0.0,,
P5,0.34,0.14,0.28,0.28,0.0,
P6,0.24,0.24,0.1,0.22,0.39,0.0



Iteration: 1

Distance between P3 and P6 is smallest (0.1)

Merge P3 and P6 into a single cluster

Using average linkage, the distance matrix is


,P1,P2,"(P3, P6)",P4,P5
P1,0,,,,
P2,0.23,0,,,
"(P3, P6)",0.23,0.19,0,,
P4,0.37,0.19,0.19,0,
P5,0.34,0.14,0.34,0.28,0


-----------------------------------------------------------------

Iteration: 2

Distance between P2 and P5 is smallest (0.14)

Merge P2 and P5 into a single cluster

Using average linkage, the distance matrix is


...

-----------------------------------------------------------------

Iteration: 4

Distance between ('P2', 'P5') and (('P3', 'P6'), 'P4') is smallest (0.25)

Merge ('P2', 'P5') and (('P3', 'P6'), 'P4') into a single cluster

Using average linkage, the distance matrix is


,P1,"((P2, P5), ((P3, P6), P4))"
P1,0,
"((P2, P5), ((P3, P6), P4))",0.3,0


-----------------------------------------------------------------

Iteration: 5

Distance between P1 and (('P2', 'P5'), (('P3', 'P6'), 'P4')) is smallest (0.3)

Merge P1 and (('P2', 'P5'), (('P3', 'P6'), 'P4')) into a single cluster

Using average linkage, the distance matrix is


,"(P1, ((P2, P5), (('P3', 'P6'), P4)))"
"(P1, ((P2, P5), (('P3', 'P6'), P4)))",0


-----------------------------------------------------------------
